In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *

### Collaborative Filtering Deep Dive

Collaborative filtering usually calls thing *items* rather than *products*

Key foundational idea is *latent factors* 

#### A First Look at the Data

We're going to use a dataset called [MovieLens](https://grouplens.org/datasets/movielens/)

- Contains 10s of millions of rankings(a combo of a movie id, user id, and numeric rating)

- Just going to use on a subset of 100k for this

- Good practice to go back and try this on the full set at some point

In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [8]:
path.ls()

(#276) [Path('/home/petewin/.fastai/data/ml-100k/u2.base'),Path('/home/petewin/.fastai/data/ml-100k/u.genre'),Path('/home/petewin/.fastai/data/ml-100k/u.user'),Path('/home/petewin/.fastai/data/ml-100k/u3.test'),Path('/home/petewin/.fastai/data/ml-100k/ub.test'),Path('/home/petewin/.fastai/data/ml-100k/u.occupation'),Path('/home/petewin/.fastai/data/ml-100k/ua.test'),Path('/home/petewin/.fastai/data/ml-100k/u5.base'),Path('/home/petewin/.fastai/data/ml-100k/u4.test'),Path('/home/petewin/.fastai/data/ml-100k/u.info')...]

In [10]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Above has everything we need, there's also cross-tabulated data that's more human-readable

In the cross-tab format we can see that users have rated some movies, not others, we're going to try to predict how much they would enjoy a given movie that they have not yet rated

In [11]:
last_skywalker = np.array([0.98, 0.9, -0.9])

In [12]:
user1 = np.array([0.9, 0.8, -0.6])

In [13]:
(user1 * last_skywalker).sum()

2.1420000000000003

When multiplying two vectors together and adding up results, it's known as the *dot product*

In [14]:
casablanca = np.array([-0.99, -0.3, 0.8])

In [15]:
(user1 * casablanca).sum()

-1.611

In [17]:
user1 * casablanca

array([-0.891, -0.24 , -0.48 ])

In [18]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()                     

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Merge with ratings table

In [19]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [30]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,90,Arsenic and Old Lace (1944),4
1,770,Face/Off (1997),4
2,881,North by Northwest (1959),4
3,13,Victor/Victoria (1982),1
4,774,"Graduate, The (1967)",1
5,798,Ghost (1990),3
6,294,"Lost World: Jurassic Park, The (1997)",4
7,498,"NeverEnding Story III, The (1994)",2
8,75,Executive Decision (1996),4
9,178,"Usual Suspects, The (1995)",5


In [34]:
dls_item_test = CollabDataLoaders.from_df(ratings, bs=64)
dls.show_batch()

,user,title,rating
0,666,Scream (1996),3
1,128,My Fair Lady (1964),3
2,385,"Fugitive, The (1993)",3
3,894,"Secret of Roan Inish, The (1994)",4
4,434,"First Wives Club, The (1996)",4
5,500,"African Queen, The (1951)",4
6,188,Primal Fear (1996),5
7,897,"Indian in the Cupboard, The (1995)",3
8,764,Schindler's List (1993),5
9,514,Multiplicity (1996),3


In [37]:
dls_item_test.x_names

(#2) ['user','movie']

In [38]:
dls.x_names

(#2) ['user','title']

In [67]:
n_factors_pb = np.argmax(ratings['rating'])
n_factors_pb = ratings.iloc[n_factors_pb]['rating']
n_factors_pb

5

In [57]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

# generate intial user and movie factors matrices
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [58]:
n_users, n_movies

(944, 1665)

In [84]:
user_factors, user_factors.shape, user_factors[2]

(tensor([[ 1.3450, -0.5776,  2.6283, -0.7433,  1.5435],
         [ 1.5770,  0.3048, -0.8704,  0.3392,  1.0310],
         [ 0.3728,  1.0886,  1.1089,  1.6928,  0.7272],
         ...,
         [-0.0157, -1.1696, -0.6434,  0.0856,  0.3593],
         [-0.8464, -0.4229,  1.5013,  0.1527, -2.2702],
         [ 0.4366,  1.1595,  0.2274, -0.7203, -1.4122]]),
 torch.Size([944, 5]),
 tensor([-1.7154,  0.6849,  1.3640,  1.6230,  0.8983]))

In [60]:
movie_factors, movie_factors.shape

(tensor([[ 1.4293,  1.1051,  0.9125,  0.3597,  0.9982],
         [-0.6449,  0.0367,  3.4299,  0.4531,  0.6598],
         [-0.6240,  0.3590, -0.7743,  0.4714,  0.2899],
         ...,
         [ 0.7184, -0.4105,  0.3172, -0.6800,  0.5724],
         [-0.9471, -0.3746, -0.3306, -0.6293, -0.7928],
         [-2.1265,  1.0739,  0.3538, -0.2682, -0.1980]]),
 torch.Size([1665, 5]))

Deep learning models don't know how to "look up at index" to check the corresponding index values between a movie latent factor matrix and a user latent factor matrix. If it could we could do the *dot product* of the two values to compare

The workaround is to represent *look up at an index* as a matrix product.

Trick is to replace our indices with *one-hot encoded* vectors

In [89]:
one_hot_2 = one_hot(2, n_users).float()
one_hot_2, one_hot_2.shape

(tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [90]:
user_factors.t(), user_factors

(tensor([[ 1.3450,  1.5770,  0.3728,  ..., -0.0157, -0.8464,  0.4366],
         [-0.5776,  0.3048,  1.0886,  ..., -1.1696, -0.4229,  1.1595],
         [ 2.6283, -0.8704,  1.1089,  ..., -0.6434,  1.5013,  0.2274],
         [-0.7433,  0.3392,  1.6928,  ...,  0.0856,  0.1527, -0.7203],
         [ 1.5435,  1.0310,  0.7272,  ...,  0.3593, -2.2702, -1.4122]]),
 tensor([[ 1.3450, -0.5776,  2.6283, -0.7433,  1.5435],
         [ 1.5770,  0.3048, -0.8704,  0.3392,  1.0310],
         [ 0.3728,  1.0886,  1.1089,  1.6928,  0.7272],
         ...,
         [-0.0157, -1.1696, -0.6434,  0.0856,  0.3593],
         [-0.8464, -0.4229,  1.5013,  0.1527, -2.2702],
         [ 0.4366,  1.1595,  0.2274, -0.7203, -1.4122]]))

In [87]:
user_factors.t().shape, user_factors.shape

(torch.Size([5, 944]), torch.Size([944, 5]))

In [88]:
user_factors.t()

tensor([[ 1.3450,  1.5770,  0.3728,  ..., -0.0157, -0.8464,  0.4366],
        [-0.5776,  0.3048,  1.0886,  ..., -1.1696, -0.4229,  1.1595],
        [ 2.6283, -0.8704,  1.1089,  ..., -0.6434,  1.5013,  0.2274],
        [-0.7433,  0.3392,  1.6928,  ...,  0.0856,  0.1527, -0.7203],
        [ 1.5435,  1.0310,  0.7272,  ...,  0.3593, -2.2702, -1.4122]])

This:

In [91]:
user_factors.t() @ one_hot_2

tensor([0.3728, 1.0886, 1.1089, 1.6928, 0.7272])

Gives us the same result as this:

In [92]:
user_factors[2]

tensor([0.3728, 1.0886, 1.1089, 1.6928, 0.7272])

Doing it this way would be fine but storing and searching that one-hot encoded vector is slow and hogs memory, ideally we should be able to directly index into an array using an integer

So most libraries, including pytorch provide a way to do this OOTB

- There is a special layer that does this, it indexes a vector using an integer
- But it does by calculating its derivative in such a way that it's identical to if we actually did the above one hot matrix multiplication

**This is called an embedding**

The thing you multiply the one-hot encoded matrix by in this case is called the *embedding matrix*

### Collaborative Filtering from Scratch